# Automatic Blog Generator

In this project, we will be  combining OpenAi with a document a page the lives on the internet(GitHub).

<h4>Setup Github Account</h4>
<ul>
    <li>Create a new public repository on github using the format: username.github.io</li>
 <li>Clone the repository</li>
 <li>Create an index.html file</li>
 <li>Add some content to the index file</li>
 <li>Push changes made and go to https://username.github.io to view document on the web</li>
    </ul>


In [1]:
import openai
import os

In [2]:
from getpass import getpass
openai.api_key=getpass()

········


In [3]:
#Install the GitPython library. It allows us to make git calls within python code
!pip3 install GitPython

In [84]:
# Import Repo(Git repository) and set repo path as variables
from git import Repo

In [85]:
#Import Path functionality-This allows us to easily set paths
from pathlib import Path

In [86]:
pwd

'/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io'

In [87]:
PATH_TO_BLOG_REPO=Path('/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io/.git')

In [88]:
PATH_TO_BLOG_REPO.parent

PosixPath('/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io')

In [89]:
PATH_TO_BLOG=PATH_TO_BLOG_REPO.parent

In [90]:
#Create a sub directory called content
PATH_TO_BLOG/"content"

PosixPath('/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io/content')

In [91]:
#Asign a variable to the path to blog
PATH_TO_CONTENT= PATH_TO_BLOG/"content"

In [92]:
#There is actually no content folder yet, so lets create one
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

# Function to automatically update web page 

Now that we've established our file path, let's verify that we can utilize Git to automate updates and push changes to the blog. To achieve this, we'll create a function that can handle adding, committing, and pushing changes from within Python

In [13]:
# Function to update blog
def update_blog(commit_message='The Blog has just been updated'):
    #Sync to a reposotory using the gitPython library Repo
    repo=Repo(PATH_TO_BLOG_REPO)
    #Call the git add command to add new update vis the repo path
    repo.git.add(all=True)
    #git commit -m "The Blog has just been updated"
    repo.index.commit(commit_message)
    #git push
    origin = repo.remote(name='origin')
    origin.push()
    

In [14]:
#let us check if this works using a random text to uodate the blog post
randon_text_string ="Just testing here, note that this will replace the original content"

In [15]:
with open(PATH_TO_BLOG/"index.html", 'w') as f:
    f.write(randon_text_string)

In [16]:
update_blog()

# Function to automatically create HTML pages

The function will have the ability to generate HTML pages automatically and complete with placeholders for text and images. These placeholders will later be populated with data retrieved from OpenAI API calls. The pages will be pushed to the content directory created above

In [75]:
#Function to create new blog pages
#import the shutil utility to help with copying and moving files, as well as archiving utilities.

import shutil
def create_new_blog(title, content, cover_image):
    cover_image =Path(cover_image)
    #Get a list of files and count the number of html files in the content directory
    files =len(list(PATH_TO_CONTENT.glob(".html")))
    new_title = f"(files+1).html"
    path_to_new_content = PATH_TO_CONTENT/new_title
    
    #copy cover image and put it in the path to content
    
    shutil.copy(cover_image,PATH_TO_CONTENT)
    #If file already exist. If already exist raise an error message
    if not os.path.exists(path_to_new_content):
        #Write a new html file
        with open(path_to_new_content, 'w') as f:
            f.write('<!DOCTYPE HTML >\n')
            f.write("<html>\n")
            f.write("<head>\n")
            f.write(f"<title> {title} </title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}'  alt='Cover image'> <br />\n")
            f.write(f"<h1>{title} </h1>")
            
            #Introduce OpenAI here
            f.write(content.replace("\n", "<br />\n") )
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog has been created")
            return path_to_new_content
   # else:
    #        raise FileExistsError('This file already exsist, please check the name of the file! Aborting....')


In [76]:
path_to_new_content = create_new_blog('Test_title','Randon text','accenture.svg')

We will automate the process of adding a link to all blog posts on the index page using the library known as Beautiful Soup. This will involve creating a list on the index page that contains links to all blog posts

In [77]:
#Install beautifulsoup4: used for web scraping purposes to extract data from HTML and XML documents. It provides a simple and efficient way to navigate, search, and modify the parsed HTML or XML data.
!pip install beautifulsoup4

In [78]:
from bs4 import BeautifulSoup as Soup

In [79]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [80]:
str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport"/>\n<title>Document</title>\n</head>\n<body>\n<h1> My Blockchain Blog</h1>\n<a href="index.html">Home</a>\n</body>\n</html>'

Now let us check for duplicate links and pass in the path to the new content

In [93]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls

Now lets write to the index file

In [94]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/"index.html") as index:
        soup = Soup(index.read())

    links = soup.find_all("a")
    last_link = links[-1]
    
    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("Link does already exist!")
        
    link_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    link_to_new_blog.string = path_to_new_content.name.split(".")[0]
    last_link.insert_after(link_to_new_blog)
    
    with open(PATH_TO_BLOG/"index.html", "w") as f:
        f.write(str(soup.prettify(formatter='html')))

In [95]:
write_to_index(path_to_new_content)

AttributeError: 'NoneType' object has no attribute 'parts'